In [24]:
#importing required libraries

import tweepy
import json
import time
from dotenv import load_dotenv
import boto3
import os
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\p_k_s\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\p_k_s\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\p_k_s\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\p_k_s\AppData\Roaming\nltk_data...


In [2]:
# Load the environment variables from .env file
load_dotenv()

# Get the Bearer token value from the environment variable
bearer_token = os.getenv('BEARER_TOKEN')

In [8]:
# Authenticate with Twitter API using API keys, API secret key, and Bearer token
auth = tweepy.OAuth2BearerHandler(bearer_token)
api = tweepy.API(auth)

# Define the search query
query = 'covid'

# Define the fields to include in the response
fields = ['created_at', 'id', 'full_text', 'user']

# Extract data using search API
tweets = []
start_time = time.time()
seconds = 1800
while True:
    elapsed_time = time.time() - start_time
    if elapsed_time >= seconds:
        break
    try:
        for status in tweepy.Cursor(api.search_tweets,
                                    q=query,
                                    tweet_mode='extended',
                                    lang='en',
                                    result_type='recent',
                                    count=100).items():
             # Create a new dictionary with only the desired fields
            tweet_data = {field: status._json[field] for field in fields}
            tweets.append(tweet_data)
            #print(elapsed_time)
            #time.sleep(60) # Wait for 60 seconds and try again
    except Exception as e:
        print(f"Error occurred: {e}")
        time.sleep(30) # Wait for 60 seconds and try again

# Save the extracted data as JSON
with open('tweets.json', 'w') as f:
    json.dump(tweets, f)

# Print confirmation message
print(f"Saved {len(tweets)} tweets to tweets.json")

Error occurred: 429 Too Many Requests
88 - Rate limit exceeded
Error occurred: 429 Too Many Requests
88 - Rate limit exceeded
Error occurred: 429 Too Many Requests
88 - Rate limit exceeded
Error occurred: 429 Too Many Requests
88 - Rate limit exceeded
Error occurred: 429 Too Many Requests
88 - Rate limit exceeded
Error occurred: 429 Too Many Requests
88 - Rate limit exceeded
Error occurred: 429 Too Many Requests
88 - Rate limit exceeded
Error occurred: 429 Too Many Requests
88 - Rate limit exceeded
Error occurred: 429 Too Many Requests
88 - Rate limit exceeded
Error occurred: 429 Too Many Requests
88 - Rate limit exceeded
Error occurred: 429 Too Many Requests
88 - Rate limit exceeded
Error occurred: 429 Too Many Requests
88 - Rate limit exceeded
Error occurred: 429 Too Many Requests
88 - Rate limit exceeded
Error occurred: 429 Too Many Requests
88 - Rate limit exceeded
Error occurred: 429 Too Many Requests
88 - Rate limit exceeded
Error occurred: 429 Too Many Requests
88 - Rate limit e

In [10]:
# assuming `tweets` is the list of dictionaries
df = pd.DataFrame(tweets, columns=['id', 'created_at', 'full_text'])


In [11]:
# printing the DataFrame
df.head()

,id,created_at,full_text
0,1634613559543083009,Sat Mar 11 17:53:36 +0000 2023,RT @CitizenFreePres: One thing I've learned ab...
1,1634613559484137474,Sat Mar 11 17:53:36 +0000 2023,RT @bambkb: Everything could cause death and i...
2,1634613556963430406,Sat Mar 11 17:53:36 +0000 2023,RT @Jim_Jordan: The House just voted unanimous...
3,1634613555491377152,Sat Mar 11 17:53:35 +0000 2023,Don't forget we are offering FREE flu and COVI...
4,1634613554610601984,Sat Mar 11 17:53:35 +0000 2023,"@ntsikasaunty Wow, he has lost a lot of weight..."


In [26]:
# Define a function to clean tweets
def clean_tweet(tweet):
    # Remove Twitter-specific entities
    tweet = re.sub(r'@[A-Za-z0-9_]+', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tweet = re.sub(r'https?:\/\/\S+', '', tweet)

    # Remove punctuation and special characters
    tweet = re.sub(r'[^\w\s]', '', tweet)

    # Tokenize the tweet
    tokens = word_tokenize(tweet)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if not token.lower() in stop_words]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    # Convert tokens to lowercase and join them into a string
    cleaned_tweet = " ".join([token.lower() for token in lemmatized_tokens])

    return cleaned_tweet


# Clean the tweets
df['cleaned_tweet'] = df['full_text'].apply(clean_tweet)

# Initialize the VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Compute the sentiment scores for each tweet
df['sentiment_scores'] = df['cleaned_tweet'].apply(lambda x: sia.polarity_scores(x))

# Map the sentiment scores to categories
def map_sentiment_score_to_category(sentiment_score):
    if sentiment_score >= 0.6:
        return 'very good'
    elif sentiment_score >= 0.2:
        return 'good'
    elif sentiment_score > -0.2:
        return 'neutral'
    elif sentiment_score > -0.6:
        return 'bad'
    else:
        return 'very bad'

df['sentiment_category'] = df['sentiment_scores'].apply(lambda x: map_sentiment_score_to_category(x['compound']))

# Print the results
print(df[['full_text', 'sentiment_category']])

                                               full_text sentiment_category
0      RT @CitizenFreePres: One thing I've learned ab...          very good
1      RT @bambkb: Everything could cause death and i...           very bad
2      RT @Jim_Jordan: The House just voted unanimous...               good
3      Don't forget we are offering FREE flu and COVI...           very bad
4      @ntsikasaunty Wow, he has lost a lot of weight...               good
...                                                  ...                ...
89985  Theresa Tam and all Public Health Officers &am...           very bad
89986  @DonaldWelsh16 Indeed, the long-term problems ...           very bad
89987  RT @EnemyInAState: Previous research has sugge...                bad
89988  @elonmusk @RepMattGaetz @Timcast Someone doesn...                bad
89989  TrustHealthInf: Frequently asked questions on ...           very bad

[89990 rows x 2 columns]


In [27]:
df.head()

,id,created_at,full_text,cleaned_tweet,sentiment_scores,sentiment_category
0,1634613559543083009,Sat Mar 11 17:53:36 +0000 2023,RT @CitizenFreePres: One thing I've learned ab...,rt one thing ive learned guy last 20 year love...,"{'neg': 0.0, 'neu': 0.756, 'pos': 0.244, 'comp...",very good
1,1634613559484137474,Sat Mar 11 17:53:36 +0000 2023,RT @bambkb: Everything could cause death and i...,rt everything could cause death injury except ...,"{'neg': 0.456, 'neu': 0.544, 'pos': 0.0, 'comp...",very bad
2,1634613556963430406,Sat Mar 11 17:53:36 +0000 2023,RT @Jim_Jordan: The House just voted unanimous...,rt house voted unanimously declassify intellig...,"{'neg': 0.0, 'neu': 0.744, 'pos': 0.256, 'comp...",good
3,1634613555491377152,Sat Mar 11 17:53:35 +0000 2023,Don't forget we are offering FREE flu and COVI...,dont forget offering free flu covid19 vaccinat...,"{'neg': 0.324, 'neu': 0.604, 'pos': 0.072, 'co...",very bad
4,1634613554610601984,Sat Mar 11 17:53:35 +0000 2023,"@ntsikasaunty Wow, he has lost a lot of weight...",wow lost lot weight covid something else,"{'neg': 0.207, 'neu': 0.45, 'pos': 0.342, 'com...",good
